In [1]:
#nbi:hide_in
import pandas as pd
import numpy as np
import scipy as sp
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
#plt.rcParams["figure.dpi"]=160 #This is for specifying the dpi and size of the graph
plt.rcParams['figure.figsize']=(20,10)
plt.rcParams['axes.grid'] = True

In [2]:
#nbi:hide_in
def IMAC_R(city):
    data=pd.read_csv('Raw_Data_Cities.csv')
    complete_date=pd.date_range(start='12-01-2021 00:00',end='12-31-2022 23:00',freq='H')
    data=data.assign(Time=complete_date[744:])
    dec_vals=data.iloc[-744:][city]
    d = {'Time':complete_date[:744], city: dec_vals}
    december_df=pd.DataFrame(data=d)
    Complete_df=pd.concat((december_df,data))
    Complete_df['month']=pd.to_datetime(Complete_df['Time']).dt.month
    Complete_df['day']=pd.to_datetime(Complete_df['Time']).dt.day
    Complete_df['year']=pd.to_datetime(Complete_df['Time']).dt.year
    average_daily=Complete_df.groupby(['year','month','day'])[city].mean()
    rolling_mean_30=np.round(average_daily.rolling(30).mean(),2)
    w,x,=np.array(np.round(average_daily.iloc[31:],2)),np.array(rolling_mean_30.iloc[31:])
    dd=pd.date_range(start='01-01-2022',end='12-31-2022',freq='D')
    dday=dd.day
    dmon=dd.month
    T_neutral=np.round(x*0.42+17.6,2)
    T_plus=T_neutral+2.15
    T_minus=T_neutral-2.15
    numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
    Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Average drybulb temperature outside','Rolling_mean','Monthly Average Neutral Temp.','90% acceptability upper band','90% acceptability lower band'])
    x=np.round(Final_Data.groupby('Month').mean(),1)
    y=x.drop(['Day','Rolling_mean'],axis=1)
    return y

In [3]:
#nbi:hide_in
def NV(T):
    return 0.54*T+12.83
def MM(T):
    return 0.28*T+17.87
def AC(T):
    return 0.05*T+23.96

In [4]:
#nbi:hide_in
def IMAC(city,building='all'):
    data=pd.read_csv('Raw_Data_Cities.csv')
    complete_date=pd.date_range(start='12-01-2021 00:00',end='12-31-2022 23:00',freq='H')
    data=data.assign(Time=complete_date[744:])
    dec_vals=data.iloc[-744:][city]
    d = {'Time':complete_date[:744], city: dec_vals}
    december_df=pd.DataFrame(data=d)
    Complete_df=pd.concat((december_df,data))
    Complete_df['month']=pd.to_datetime(Complete_df['Time']).dt.month
    Complete_df['day']=pd.to_datetime(Complete_df['Time']).dt.day
    Complete_df['year']=pd.to_datetime(Complete_df['Time']).dt.year
    average_daily=Complete_df.groupby(['year','month','day'])[city].mean()
    rolling_mean_30=np.round(average_daily.rolling(30).mean(),2)
    w,x,=np.array(np.round(average_daily.iloc[31:],2)),np.array(rolling_mean_30.iloc[31:])
    dd=pd.date_range(start='01-01-2022',end='12-31-2022',freq='D')
    dday=dd.day
    dmon=dd.month
    if building=='NV':
        T_neutral=np.round(NV(x),1)
        T_plus=T_neutral+2.4
        T_minus=T_neutral-2.4
        numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
        Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Average drybulb temperature outside','Rolling_mean','Monthly Average Neutral Temp.','90% acceptability upper band','90% acceptability lower band'])
        x=np.round(Final_Data.groupby('Month').mean(),1)
        y=x.drop(['Day','Rolling_mean'],axis=1)
        return y
    elif building=='MM':
        T_neutral=np.round(MM(x),1)
        T_plus=T_neutral+3.5
        T_minus=T_neutral-3.5
        numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
        Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Average drybulb temperature outside','Rolling_mean','Monthly Average Neutral Temp.','90% acceptability upper band','90% acceptability lower band'])
        x=np.round(Final_Data.groupby('Month').mean(),1)
        y=x.drop(['Day','Rolling_mean'],axis=1)
        return y
    elif building=='AC':
        T_neutral=np.round(AC(x),1)
        T_plus=T_neutral+2.1
        T_minus=T_neutral-2.1
        numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
        Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Average drybulb temperature outside','Rolling_mean','Monthly Average Neutral Temp.','90% acceptability upper band','90% acceptability lower band'])
        x=np.round(Final_Data.groupby('Month').mean(),1)
        y=x.drop(['Day','Rolling_mean'],axis=1)
        return y
    elif building=='all':
        T_neutral_NV=np.round(NV(x),1)
        T_neutral_MM=np.round(MM(x),1)
        T_neutral_AC=np.round(AC(x),1)
        T_plus_NV=T_neutral_NV+2.4
        T_plus_MM=T_neutral_MM+3.5
        T_plus_AC=T_neutral_AC+2.1
        T_minus_NV=T_neutral_NV-2.4
        T_minus_MM=T_neutral_MM-3.5
        T_minus_AC=T_neutral_AC-2.1
        lst=(dmon,dday,w,x,T_neutral_NV,T_plus_NV,T_minus_NV,T_neutral_MM,T_plus_MM,T_minus_MM,T_neutral_AC,T_plus_AC,T_minus_AC)
        columns=['Month','Day','Average drybulb temperature outside','Rolling_mean','NV Neutral','NV 90% acceptability upper band','NV 90% acceptability lower band','MM Neutral','MM 90% acceptability upper band','MM 90% acceptability lower band','AC Neutral','AC 90% acceptability upper band','AC 90% acceptability lower band']
        numpy_array=(np.column_stack(lst))
        Final_Data=pd.DataFrame(numpy_array,columns=columns)
        x=np.round(Final_Data.groupby('Month').mean(),1)
        y=x.drop(['Day','Rolling_mean'],axis=1)
        return y
    

In [5]:
def graph_IMAC_R(city):
    j=(np.array(IMAC_R(city).index))
    x1=j.astype(int)
    y1=IMAC_R(city)['Average drybulb temperature outside']
    y2=IMAC_R(city)['Monthly Average Neutral Temp.']
    y3=IMAC_R(city)['90% acceptability upper band']
    y4=IMAC_R(city)['90% acceptability lower band']
    plt.plot(x1,y1,ls='--',c='k',label='Average Outdoor Dry Bulb Temp.')
    plt.plot(x1,y2,ls='-',c='goldenrod',label='Monthly Average Neutral Temp. (Indoor Operations)')
    #plt.plot(x1,y3,ls='-',c='firebrick')
    #plt.plot(x1,y4,ls='-',c='firebrick')
    plt.xlabel('Month',loc='center')
    plt.ylabel('Temperature $^{\circ}C$',loc='center')
    plt.fill_between(x1,y3,y4,color='wheat',alpha=0.7,label='90% Acceptability Range (Indoor Operations)')
    plt.grid(alpha=0.3)
    labels=['Jan','Feb','Mar','Apr','May','Jun','July','Aug','Sep','Oct','Nov','Dec']
    plt.xticks(x1, labels, rotation='vertical')
    plt.xlim(1,12)
    plt.title('IMAC(R):Monthly Comfortable Temperatures')
    plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.22,1,1),bbox_transform = plt.gcf().transFigure )
    plt.show()

In [6]:
def graph_IMAC(city,building):
    if building!='all':
        j=(np.array(IMAC_R(city).index))
        x1=j.astype(int)
        y1=IMAC(city,building)['Average drybulb temperature outside']
        y2=IMAC(city,building)['Monthly Average Neutral Temp.']
        y3=IMAC(city,building)['90% acceptability upper band']
        y4=IMAC(city,building)['90% acceptability lower band']
        plt.plot(x1,y1,ls='--',c='k',label='Average Outdoor Dry Bulb Temp.')
        plt.plot(x1,y2,ls='-',c='goldenrod',label='Monthly Average Neutral Temp. (Indoor Operations)')
        plt.xlabel('Month',loc='center')
        plt.ylabel('Temperature $^{\circ}C$',loc='center')
        plt.fill_between(x1,y3,y4,color='wheat',alpha=0.7,label='90% Acceptability Range (Indoor Operations)')
        plt.grid(alpha=0.3)
        labels=['Jan','Feb','Mar','Apr','May','Jun','July','Aug','Sep','Oct','Nov','Dec']
        plt.xticks(x1, labels, rotation='vertical')
        plt.xlim(1,12)
        plt.title('IMAC:Monthly Comfortable Temperatures '+'for '+str(building))
        plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.22,1,1),bbox_transform = plt.gcf().transFigure )
        plt.show()
    else:
        fig,ax=plt.subplots(3,sharex=True,sharey=True)
        j=(np.array(IMAC_R(city).index))
        x1=j.astype(int)
        y1=IMAC(city,'all')['Average drybulb temperature outside']
        y2=IMAC(city,'all')['NV Neutral']
        y3=IMAC(city,'all')['NV 90% acceptability upper band']
        y4=IMAC(city,'all')['NV 90% acceptability lower band']
        y5=IMAC(city,'all')['MM Neutral']
        y6=IMAC(city,'all')['MM 90% acceptability upper band']
        y7=IMAC(city,'all')['MM 90% acceptability lower band']
        y8=IMAC(city,'all')['AC Neutral']
        y9=IMAC(city,'all')['AC 90% acceptability upper band']
        y10=IMAC(city,'all')['AC 90% acceptability lower band']
        ax[0].plot(x1,y1,ls='--',c='k',label='Average Outdoor Dry Bulb Temp.')
        ax[0].plot(x1,y2,ls='-',c='goldenrod',label='Monthly Average Neutral Temp. (Indoor Operations)')
        ax[0].fill_between(x1,y3,y4,color='wheat',alpha=0.7,label='90% Acceptability Range (Indoor Operations)')

        ax[1].plot(x1,y1,ls='--',c='k',label='Average Outdoor Dry Bulb Temp.')
        ax[1].plot(x1,y5,ls='-',c='goldenrod',label='Monthly Average Neutral Temp. (Indoor Operations)')
        ax[1].fill_between(x1,y6,y7,color='wheat',alpha=0.7,label='90% Acceptability Range (Indoor Operations)')

        ax[2].plot(x1,y1,ls='--',c='k',label='Average Outdoor Dry Bulb Temp.')
        ax[2].plot(x1,y8,ls='-',c='goldenrod',label='Monthly Average Neutral Temp. (Indoor Operations)')
        ax[2].fill_between(x1,y9,y10,color='wheat',alpha=0.7,label='90% Acceptability Range (Indoor Operations)')

        ax[0].set_title('Naturally Ventillated')
        ax[1].set_title('Mix Mode')
        ax[2].set_title('Air Conditioned')

        labels=['Jan','Feb','Mar','Apr','May','Jun','July','Aug','Sep','Oct','Nov','Dec']
        plt.setp(ax,xticks=[1,2,3,4,5,6,7,8,9,10,11,12],xticklabels=labels,yticks=[20,25,30,35],xlim=(1,12))
        fig.supxlabel('Month')
        fig.supylabel('Temperature $^{\circ}C$')
        fig.tight_layout()
        plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.22,1,1),bbox_transform = plt.gcf().transFigure )
        plt.show()    

In [7]:
#nbi:hide_in
b_type=widgets.Dropdown(
    options=[('Naturally Ventilated','NV'), ('Mixed Mode','MM'), ('Air Conditioned','AC'),('All','all')],
    value='NV',
    description='Building:',)
city_list=widgets.Dropdown(
    #value='Ahmedabad',
    placeholder='Type your city',
    options=['Ahmedabad','Amritsar','Aurangabad','Akola','Allahabad','Belgaum','Bengaluru','Bhopal','Bhubaneshwar','Bhuj','Barmer','Bhagalpur','Bikaner','Chennai','Chitradurga','Dehradun','Dibrugarh','Gorakhpur','Guwahati','Gwalior','Hisar','Hyderabad','Imphal','Indore','Jaipur','Jaisalmer','Jodhpur','Jabalpur','Jagdalpur','Jamnagar','Jorhat','Kolkata','Kurnool','Kota','Lucknow','Mumbai','Mangalore','Nagpur','Nellore','New Delhi','Panjim','Patna','Pune','Rajkot','Ratnagiri','Raipur','Ramagundam','Ranchi','Raxaul','Solapur','Saharanpur','Shilong','Surat','Thiruvananthapuram','Tiruchchirapalli','Tezpur','Varanasi','Veraval','Vishakhapatnam'],
    description='City:',
    ensure_option=True,
    disabled=False
)
get_data_button_1=widgets.Button(description='Get Data')
get_data_button_2=widgets.Button(description='Get Data')

In [8]:
#nbi:hide_in
out = widgets.Output()
out1=widgets.Output()

In [9]:
#nbi:hide_in
def on_button_clicked_1(b):
    with out:
        clear_output()
        display(IMAC(city_list.value,building=b_type.value))
    with out1: 
        clear_output()
        (graph_IMAC(city_list.value,building=b_type.value))

get_data_button_1.on_click(on_button_clicked_1)

def on_button_clicked_2(b):
    with out:
        clear_output()
        display(IMAC_R(city_list.value))
    with out1:
        clear_output()
        (graph_IMAC_R(city_list.value))
get_data_button_2.on_click(on_button_clicked_2)

In [10]:
#nbi:hide_in
box1 = widgets.VBox([city_list,b_type,get_data_button_1,out])
box2=widgets.VBox([city_list,get_data_button_2,out])

In [11]:
#nbi:hide_in
tab_nest = widgets.Tab()
tab_nest.children = [box1, box2]
tab_nest.set_title(0, 'Commercial Space')
tab_nest.set_title(1, 'Residential Space')

In [12]:
footer = widgets.HTML("<h2>Calculating comfortable temperatures for your city</h2>")
header= widgets.HTML('<div id="img" style="centered">'
                    '<img style="centered" src="footer.png" alt="logo">'                       
                    '</div>')
header.style.text_align='center'
widgets.AppLayout(header=header,left_sidebar=(tab_nest),center=out1,right_sidebar=None,footer=None)

AppLayout(children=(HTML(value='<div id="img" style="centered"><img style="centered" src="footer.png" alt="log…

In [13]:
plt.rcParams.keys

<bound method Mapping.keys of RcParams({'_internal.classic_mode': False,
          'agg.path.chunksize': 0,
          'animation.bitrate': -1,
          'animation.codec': 'h264',
          'animation.convert_args': [],
          'animation.convert_path': 'convert',
          'animation.embed_limit': 20.0,
          'animation.ffmpeg_args': [],
          'animation.ffmpeg_path': 'ffmpeg',
          'animation.frame_format': 'png',
          'animation.html': 'none',
          'animation.writer': 'ffmpeg',
          'axes.autolimit_mode': 'data',
          'axes.axisbelow': 'line',
          'axes.edgecolor': 'black',
          'axes.facecolor': 'white',
          'axes.formatter.limits': [-5, 6],
          'axes.formatter.min_exponent': 0,
          'axes.formatter.offset_threshold': 4,
          'axes.formatter.use_locale': False,
          'axes.formatter.use_mathtext': False,
          'axes.formatter.useoffset': True,
          'axes.grid': True,
          'axes.grid.axis': 'both',